In [1]:
import glob
import os
import numpy as np
import scipy.io as sio

from tqdm import tqdm
from utils import *

## DEAP

In [2]:
Lab = np.zeros((0, 4))
Participant = np.zeros(0)

In [3]:
# path to change /!\

Dataset_path = '/media/storage/Dataset/Deap/EEG'

# path to change /!\

In [4]:
Feat = []
for f in tqdm(glob.glob(os.path.join(Dataset_path, '*.mat'))):
    participant_id = int(f.split('.')[0].split('/')[-1][1:])
    sig = sio.loadmat(f)
    Feat.append(feat_matrices(sig['data']).swapaxes(0,2).reshape(5, 40, -1))
    l = np.tile(sig['labels'], 16).reshape(-1, 4)
    Lab = np.concatenate((Lab, l))
    Participant = np.concatenate((Participant, participant_id*np.ones(l.shape[0])))
Feat = np.concatenate(Feat, -1)
#Normalize EEG
Feat[:, :32] = (Feat[:, :32] - Feat[:, :32].min())/np.max(Feat[:, :32] - Feat[:, :32].min())
#Normalize EMG
Feat[:, 32:34] = (Feat[:, 32:34] - Feat[:, 32:34].min())/np.max(Feat[:, 32:34] - Feat[:, 32:34].min()) 
Feat[:, 34:36] = (Feat[:, 34:36] - Feat[:, 34:36].min())/np.max(Feat[:, 34:36] - Feat[:, 34:36].min())
for i in range(36, 40):
    Feat[:, i] = (Feat[:, i] - Feat[:, i].min())/np.max(Feat[:, i] - Feat[:, i].min())    

100%|██████████| 32/32 [13:46<00:00, 25.82s/it]


In [5]:
np.save('dataset/deap/feat_eeg', Feat.swapaxes(2, 0).swapaxes(1,2)[:, :, :32])
np.save('dataset/deap/feat_phy', Feat.swapaxes(2, 0).swapaxes(1,2)[:, :, 32:])
np.save('dataset/deap/label', Lab)
np.save('dataset/deap/participant', Participant.astype(int))

## SEED - IV

In [15]:
Lab = np.zeros(0)
Participant = np.zeros(0)
Session = np.zeros(0)

In [16]:
Dataset_lab = [
    [1, 2, 3, 0, 2, 0, 0, 1, 0, 1, 2, 1, 1, 1, 2, 3, 2, 2, 3, 3, 0, 3, 0, 3],
    [2, 1, 3, 0, 0, 2, 0, 2, 3, 3, 2, 3, 2, 0, 1, 1, 2, 1, 0, 3, 0, 1, 3, 1],
    [1, 2, 2, 1, 3, 3, 3, 1, 1, 2, 1, 0, 2, 3, 3, 0, 2, 3, 0, 0, 2, 0, 1, 0]]

In [17]:
# path to change /!\

Dataset_path = '/media/storage/Dataset/Seed - IV/'

# path to change /!\

In [20]:
eeg_feat = np.zeros((62, 0, 5))
eye_feat = np.zeros((31, 0))
eye_raw = np.zeros((6, 0))
for session in range(3):
    lab = Dataset_lab[session]
    session += 1
    
    participant_files = glob.glob(os.path.join(Dataset_path, 'eeg_feature_smooth', str(session),  '*.mat'))
    
    for f in tqdm(participant_files):
        participant_id = int(f.split('/')[-1].split('_')[0])
        eeg = sio.loadmat(f)
        eye = sio.loadmat(os.path.join(Dataset_path, 'eye_feature_smooth', str(session), f.split('/')[-1]))
        
        for trial in range(len(lab)):
            eeg_feat = np.concatenate((eeg_feat, eeg['de_LDS'+str(trial+1)]), axis=1)
            eye_feat = np.concatenate((eye_feat, eye['eye_'+str(trial+1)]), axis=1)
            
            Lab = np.concatenate((Lab, lab[trial]*np.ones((eye['eye_'+str(trial+1)].shape[1]))))
            Participant = np.concatenate((Participant, participant_id*np.ones((eye['eye_'+str(trial+1)].shape[1]))))
            Session = np.concatenate((Session, (trial+1)*np.ones((eye['eye_'+str(trial+1)].shape[1]))))
            eye_r = []
            for f_ in glob.glob(os.path.join(Dataset_path, 'eye_raw_data', f.split('/')[-1].split('.')[0]+'*.mat')):
                eye_ = sio.loadmat(f_)
                for k in eye_.keys():
                    if '__' not in k:
                        eye_r.append(eye_[k].squeeze())            

            eye_raw = np.concatenate((eye_raw, ))

100%|██████████| 15/15 [00:16<00:00,  1.10s/it]


In [21]:
np.save('dataset/seed_iv/feat_eeg', eeg_feat.swapaxes(0,1).swapaxes(1,2))
np.save('dataset/seed_iv/feat_phy', eye_feat.swapaxes(0,1))
np.save('dataset/seed_iv/label', Lab.astype(int))
np.save('dataset/seed_iv/participant', Participant.astype(int))

## SEED - VIG

In [ ]:
Lab = np.zeros(0)
Participant = np.zeros(0)

In [ ]:
# path to change /!\

Dataset_path = '/media/storage/Dataset/SEED-VIG/'

# path to change /!\

In [ ]:
eeg_feat = np.zeros((17, 0, 5))
eye_feat = np.zeros((36, 0))

participant_files = glob.glob(os.path.join(Dataset_path, 'EEG_Feature_5Bands', '*.mat'))

participant_id = 1
for f in tqdm(participant_files):
    
    eeg = sio.loadmat(f)
    eye = sio.loadmat(os.path.join(Dataset_path, 'EOG_Feature', f.split('/')[-1]))
    perclos = sio.loadmat(os.path.join(Dataset_path, 'perclos_labels', f.split('/')[-1]))['perclos'].squeeze()

    eeg_feat = np.concatenate((eeg_feat, eeg['de_LDS']), axis=1)
    eye_feat = np.concatenate((eye_feat, eye['features_table_ica'].swapaxes(0,1)), axis=1)

    Lab = np.concatenate((Lab, perclos))
    Participant = np.concatenate((Participant, participant_id*np.ones((perclos.shape[0]))))
    participant_id += 1

In [ ]:
np.save('dataset/seed_vig/feat_eeg', eeg_feat.swapaxes(0,1).swapaxes(1,2))
np.save('dataset/seed_vig/feat_phy', eye_feat.swapaxes(0,1))
np.save('dataset/seed_vig/label', Lab)
np.save('dataset/seed_vig/participant', Participant.astype(int))